In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np

In [2]:
# log_data 가져오기
logdata = pd.read_csv("log_data.csv")
logdata['timestamp'] = pd.to_datetime(logdata['timestamp'])

In [3]:
# 한도 조회 과정에 해당하는 event들 중, user가 각 session에서 마지막으로 발생시킨 event만 남긴다.
# CompleteIDCertification까지 했다면, 앞의 ViewLoanApplyIntro와 StartLoanApply를 지우는 식이다.
eventdata = logdata[['user_id','event','timestamp']].sort_values(
    by=['user_id','timestamp'],ascending = True
)
# 이를 위해 한도조회에 해당하는 event들은 1,2,3의 값을 주고,
# 큰 수가 오면 바로 앞의 작은 수가 있는 event를 지우는 식으로 연산하였다.
eventdata['code'] = eventdata['event'].map(
    {'OpenApp':'OpenApp','GetCreditInfo':"GetCreditInfo",
     'UseLoanManage':"UseLoanManage",'UsePrepayCalc':"UsePrepayCalc",
     'ViewLoanApplyIntro':1,'StartLoanApply':2,
     'CompleteIDCertification':3,'UseDSRCalc':"UseDSRCalc"}
)
codelist = np.array(eventdata.code)
survlist = []
for i in range(len(codelist)-1):
    if (len(str(codelist[i])) == 1) and (len(str(codelist[i+1])) == 1):
        if (int(codelist[i+1]) > int(codelist[i])):
            survlist.append(True)
        else:
            survlist.append(False)
    else:
        survlist.append(False)
survlist.append(True)
eventdata['survive'] = survlist
eventdata = eventdata[eventdata['survive']]
del eventdata['survive'], eventdata['code'],survlist, codelist

In [5]:
# user_spec의 application 정보와 loan_result의 is_applied 정보 합치기
user_spec = pd.read_csv('user_spec.csv')
loan_result = pd.read_csv('loan_result.csv') 
user_app = pd.merge(
    user_spec[['user_id','application_id','insert_time']].drop_duplicates(
        subset='application_id'
    ),
    loan_result[['application_id','is_applied']].dropna(),
    on='application_id'
).sort_values(
    by='is_applied',ascending = False
).drop_duplicates(subset = 'application_id')

In [6]:
# is_applied 정보를 각각 'IsApplied', 'NotApplied' event로 변환
user_app['event'] = user_app['is_applied'].map(
    {1.0: "IsApplied", 0.0: "NotApplied"}
)

In [7]:
# DataFrame 정리
apply_event = user_app[['user_id','event','insert_time']].rename(
    columns = {'insert_time':'timestamp'}
).reset_index(drop = True)

In [8]:
# 다른 event들이 있는 dataframe과 합친다.
alllog = pd.concat([eventdata,apply_event],axis = 0)

In [9]:
# alllog.csv로 내보낸다.
alllog[['user_id','event','timestamp']].sort_values(
    by=['user_id','timestamp'],ascending = True
).to_csv('alllog.csv')